# **Global Terrorism Database (GTD) - Exploratory Data Analysis**

In [1]:
%config Completer.use_jedi = False

In [2]:
### importing required libraries 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from geopy.geocoders import Nominatim
import langid
import multiprocessing
from multiprocessing import Pool

In [3]:
### Setting plotting background
sns.set_style('darkgrid')
plt.rcParams['font.size'] = 11
plt.rcParams['figure.facecolor'] = '#00000000'

In [4]:
import warnings
warnings.filterwarnings("ignore")

##   **1. Data Preparation and Cleaning** 
a.   Loading the dataset into pandas dataframe <br>
b.   Checking the info. about data and columns <br>
c.   Feature selection <br>
d.   Fixing the missing & incorrect values.<br>


### a) Loading the dataset into pandas dataframe

In [7]:
### reading the csv file 
GTA_data  = pd.read_csv('/kaggle/input/gtd/globalterrorismdb_0718dist.csv',encoding = "ISO-8859-1")

In [8]:
### creating a copy of orginal data 
data = GTA_data.copy()

In [9]:
data.head()

,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,...,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,197000000001,1970,7,2,NaN,0,NaN,58,Dominican Republic,2,...,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
1,197000000002,1970,0,0,NaN,0,NaN,130,Mexico,1,...,NaN,NaN,NaN,NaN,PGIS,0,1,1,1,NaN
2,197001000001,1970,1,0,NaN,0,NaN,160,Philippines,5,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
3,197001000002,1970,1,0,NaN,0,NaN,78,Greece,8,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
4,197001000003,1970,1,0,NaN,0,NaN,101,Japan,4,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN


#### **b) Checking the info. about data and columns**

In [10]:
data.shape

(181691, 135)

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181691 entries, 0 to 181690
Columns: 135 entries, eventid to related
dtypes: float64(55), int64(22), object(58)
memory usage: 187.1+ MB


In [12]:
list(data.columns)

['eventid',
 'iyear',
 'imonth',
 'iday',
 'approxdate',
 'extended',
 'resolution',
 'country',
 'country_txt',
 'region',
 'region_txt',
 'provstate',
 'city',
 'latitude',
 'longitude',
 'specificity',
 'vicinity',
 'location',
 'summary',
 'crit1',
 'crit2',
 'crit3',
 'doubtterr',
 'alternative',
 'alternative_txt',
 'multiple',
 'success',
 'suicide',
 'attacktype1',
 'attacktype1_txt',
 'attacktype2',
 'attacktype2_txt',
 'attacktype3',
 'attacktype3_txt',
 'targtype1',
 'targtype1_txt',
 'targsubtype1',
 'targsubtype1_txt',
 'corp1',
 'target1',
 'natlty1',
 'natlty1_txt',
 'targtype2',
 'targtype2_txt',
 'targsubtype2',
 'targsubtype2_txt',
 'corp2',
 'target2',
 'natlty2',
 'natlty2_txt',
 'targtype3',
 'targtype3_txt',
 'targsubtype3',
 'targsubtype3_txt',
 'corp3',
 'target3',
 'natlty3',
 'natlty3_txt',
 'gname',
 'gsubname',
 'gname2',
 'gsubname2',
 'gname3',
 'gsubname3',
 'motive',
 'guncertain1',
 'guncertain2',
 'guncertain3',
 'individual',
 'nperps',
 'nperpcap',
 

In [13]:
""" Since the data consists of 135 feature columns I am checking the amount of missing/null values for each features which consists of 
                    more than  30% of missing values.  """
data.isna().sum()

eventid            0
iyear              0
imonth             0
iday               0
approxdate    172452
               ...  
INT_LOG            0
INT_IDEO           0
INT_MISC           0
INT_ANY            0
related       156653
Length: 135, dtype: int64

In [14]:
###  Percentage of missing values per columns
Missing_values_percent = ((data.isna().sum()/data.shape[0])*100)
Missing_values_percent.sort_values(ascending=False)

gsubname3           99.988992
weapsubtype4_txt    99.961473
weapsubtype4        99.961473
weaptype4           99.959822
weaptype4_txt       99.959822
                      ...    
suicide              0.000000
success              0.000000
crit3                0.000000
property             0.000000
eventid              0.000000
Length: 135, dtype: float64

In [15]:
print(f'The number of columns which have more than 30% of null values are:{ len(Missing_values_percent [Missing_values_percent >= 30])}')

The number of columns which have more than 30% of null values are:87


In [16]:
#### name of columns that consists of less than 30% of null values
Missing_values_percent [Missing_values_percent <= 30].index

Index(['eventid', 'iyear', 'imonth', 'iday', 'extended', 'country',
       'country_txt', 'region', 'region_txt', 'provstate', 'city', 'latitude',
       'longitude', 'specificity', 'vicinity', 'crit1', 'crit2', 'crit3',
       'doubtterr', 'multiple', 'success', 'suicide', 'attacktype1',
       'attacktype1_txt', 'targtype1', 'targtype1_txt', 'targsubtype1',
       'targsubtype1_txt', 'corp1', 'target1', 'natlty1', 'natlty1_txt',
       'gname', 'guncertain1', 'individual', 'weaptype1', 'weaptype1_txt',
       'weapsubtype1', 'weapsubtype1_txt', 'nkill', 'nwound', 'property',
       'ishostkid', 'dbsource', 'INT_LOG', 'INT_IDEO', 'INT_MISC', 'INT_ANY'],
      dtype='object')

In [ ]:
#### We are dropping all the columns that consits of more than 30% of null values.  
# perc = 30
# thresh_count =  int(((100-perc)/100)*data.shape[0] + 1)
# data.dropna(axis=1, thresh=thresh_count,inplace = True)

#### **c) Feature Selections**

In [17]:
###  Selecting only required features from the above data for further analysis 
keep_cols = ['eventid', 'iyear', 'imonth', 'iday', 'extended','country_txt','region_txt','city', 'latitude',
       'longitude','success','attacktype1_txt', 'targtype1_txt','corp1', 'target1','natlty1_txt','targsubtype1_txt',
       'gname','weaptype1_txt','nkill','nwound','property']

### creating a new dataframe with selected features 
data_new = data[keep_cols]

In [18]:
### Updating the headers
data_new.rename(columns = {'iyear':'year','imonth':'month','iday':'day','country_txt':'country_name','region_txt':'region',
                           'attacktype1_txt':'attack_type','targtype1_txt':'victim_cata','target1':'specific_victim','targsubtype1_txt':'victim_subtype_cata',
                           'corp1':'entity_name','natlty1_txt':'victim_nationaliy','gname':'perpetrator_group','weaptype1_txt':'weapon','nkill':'num_fatalities',
                           'nwound':'num_injured'},inplace = True)

In [19]:
data_new.head()

,eventid,year,month,day,extended,country_name,region,city,latitude,longitude,...,victim_cata,entity_name,specific_victim,victim_nationaliy,victim_subtype_cata,perpetrator_group,weapon,num_fatalities,num_injured,property
0,197000000001,1970,7,2,0,Dominican Republic,Central America & Caribbean,Santo Domingo,18.456792,-69.951164,...,Private Citizens & Property,NaN,Julio Guzman,Dominican Republic,Named Civilian,MANO-D,Unknown,1.0,0.0,0
1,197000000002,1970,0,0,0,Mexico,North America,Mexico city,19.371887,-99.086624,...,Government (Diplomatic),Belgian Ambassador Daughter,"Nadine Chaval, daughter",Belgium,"Diplomatic Personnel (outside of embassy, cons...",23rd of September Communist League,Unknown,0.0,0.0,0
2,197001000001,1970,1,0,0,Philippines,Southeast Asia,Unknown,15.478598,120.599741,...,Journalists & Media,Voice of America,Employee,United States,Radio Journalist/Staff/Facility,Unknown,Unknown,1.0,0.0,0
3,197001000002,1970,1,0,0,Greece,Western Europe,Athens,37.997490,23.762728,...,Government (Diplomatic),NaN,U.S. Embassy,United States,Embassy/Consulate,Unknown,Explosives,NaN,NaN,1
4,197001000003,1970,1,0,0,Japan,East Asia,Fukouka,33.580412,130.396361,...,Government (Diplomatic),NaN,U.S. Consulate,United States,Embassy/Consulate,Unknown,Incendiary,NaN,NaN,1


#### **d) Missing & incorrect values.**

In [20]:
missing_values = ((data_new.isna().sum()/data_new.shape[0])*100).sort_values(ascending= False)
missing_values

entity_name            23.418882
num_injured             8.977330
victim_subtype_cata     5.709144
num_fatalities          5.676120
longitude               2.508104
latitude                2.507554
victim_nationaliy       0.858050
specific_victim         0.350045
city                    0.238867
eventid                 0.000000
weapon                  0.000000
perpetrator_group       0.000000
attack_type             0.000000
victim_cata             0.000000
year                    0.000000
success                 0.000000
region                  0.000000
country_name            0.000000
extended                0.000000
day                     0.000000
month                   0.000000
property                0.000000
dtype: float64

In [21]:
data_new[['num_injured','num_fatalities']] = data_new[['num_injured','num_fatalities']].fillna(method='ffill')
data_new[['entity_name','specific_victim','victim_nationaliy']] =data_new[['entity_name','specific_victim','victim_nationaliy']].fillna('Unknown')
data_new['victim_subtype_cata'] = data_new['victim_subtype_cata'].fillna('Unnamed Civilian/Unspecified')

In [22]:
print(data_new[data_new.city == 'Unknown']['city'].count())
print(data_new[data_new.city == 'unknown']['city'].count())
print(len(data_new[data_new.city.isna() == True]['city']))

9775
47
434


In [23]:
data_new.city = data_new.city.replace('unknown','Unknown')
data_new['city'] = data_new['city'].fillna('Unknown')
percent_city_unknown = len(data_new[data_new.city =='Unknown'])/len(data_new)*100
print(f'The percentage of total unkown values in the city columns {percent_city_unknown}')

The percentage of total unkown values in the city columns 5.644748501576853


In [24]:
data.city.value_counts()

Unknown      9775
Baghdad      7589
Karachi      2652
Lima         2359
Mosul        2265
             ... 
Hotwag          1
Ostend          1
Balughata       1
Jikoyi          1
Kubentog        1
Name: city, Length: 36674, dtype: int64

In [25]:
unknown_city_data = data_new[(data_new['longitude'].isna()== False) & (data_new['latitude'].isna()== False) & (data_new['city'] == 'Unknown')][['longitude','latitude','city']]
unknown_city_data.head()

,longitude,latitude,city
2,120.599741,15.478598,Unknown
705,49.536102,37.146795,Unknown
1007,-6.086832,54.552254,Unknown
1085,119.583330,23.583333,Unknown
1222,121.965256,17.092014,Unknown


In [ ]:
print(f'The length of data which have unknown value for city column whose latitude and longitude is present is {len(unknown_city_data)}')

### Multiprocessing 

In [ ]:
geolocator = Nominatim(user_agent="http")

def update_city(row):
    try:
        Longitude = str(row['longitude'])
        Latitude = str(row['latitude'])
        location = geolocator.reverse(Latitude+","+Longitude, exactly_one=True)
        address = location.raw['address']
        city = address.get('city', '')

    ### Selecting the city name only in english language
        if langid.classify(city)[0] =='en':
            row['city'] = row['city'].replace('Unknown', str(city))
        else:
            row['city'] = row['city']
        return row
    
    except Exception as e:
        print(e)
        
        
def update_city2(df):
    df2 = df.apply(update_city , axis = 1)
    return df2


df1 = unknown_city_data[0:2000]
df2 = unknown_city_data[2000:4000]   
#df1 = unknown_city_data[4000:6000]
#df2 = unknown_city_data[6000:]   

if __name__ == '__main__':
    df = unknown_city_data
    p = Pool()
    result = p.map(update_city2,[df1,df2
                                 ])
    p.close()
    p.join()
    print(result)

In [ ]:
pd.concat(result,axis =0).to_csv('Geocoder_city.csv')

In [ ]:
df1 = unknown_city_data[4000:6000]
df2 = unknown_city_data[6000:]   

if __name__ == '__main__':
    df = unknown_city_data
    p = Pool()
    result = p.map(update_city2,[df1,df2
                                 ])
    p.close()
    p.join()
    print(result)

In [ ]:
pd.concat(result,axis =0).to_csv('Geocoder_city1.csv')

In [ ]:
Geocoder_data_part1 = pd.read_csv('./Geocoder_city.csv')
Geocoder_data_part2 = pd.read_csv('./Geocoder_city1.csv')

city_data = pd.concat([Geocoder_data_part1,Geocoder_data_part2],axis = 0)
city_data = city_data[(city_data.city !='Unknown') & (city_data.city.notnull())]

In [ ]:
city_data

In [ ]:
### checking out is there any missing values present in lat and long columns where city and region is not null
lat_long_missing = data_new[(data_new['longitude'].isna()== True) & 
                     (data_new['latitude'].isna()== True) & 
                     (data_new['city']!= 'Unknown') &     
                     (data_new['country_name']!= 'Unknown')][['longitude','latitude','city','country_name']]
len(lat_long_missing)

In [ ]:
lat_long_missing.head()

In [ ]:
# from geopy.geocoders import Nominatim
# geolocator = Nominatim(user_agent="my_user_agent")
# def find_lat_long(row):
#     try:
#         city = str(row['city'])
#         country = str(row['country_name'])
#         loc = geolocator.geocode(city+','+ country)
#         #loc = geolocator.geocode(country,timeout=10)
#         row['longitude'] = loc.longitude
#         row['latitude'] = loc.latitude
#         return row
#     except:
#         pass

In [ ]:
#lat_long_missing[:5].apply(find_lat_long , axis = 1)

In [ ]:
# def update_lat_long(df):
#     df_2 = df.apply(find_lat_long , axis = 1)
#     return df_2

In [ ]:
# df1 = lat_long_missing[:10]
# df2 = lat_long_missing[10:20]

# df1 = unknown_city_data[:1000]
# df2 = unknown_city_data[1000:2000]

In [ ]:
# if __name__ == '__main__':
#     df = lat_long_missing
#     p = Pool()
#     result = p.map(update_lat_long,[df1,df2
#                                  ])
#     p.close()
#     p.join()
#     print(result)
    
    
# if __name__ == '__main__':
#     df = unknown_city_data
#     p = Pool()
#     result = p.map(update_city2,[df1,df2
#                                  ])
#     p.close()
#     p.join()
#     print(result)

In [ ]:
#pd.concat(result,axis =0)

In [ ]:
#pd.concat(result,axis =0).to_csv('Geocoder_data.csv')

In [ ]:

# ### Function for finding out the city name from lattude and longitude
# def update_city(row):
#   Longitude = str(row['longitude'])
#   Latitude = str(row['latitude'])
#   location = geolocator.reverse(Latitude+","+Longitude, exactly_one=True)
#   address = location.raw['address']
#   city = address.get('state', '')

# ### Selecting the city name only in english language
#   if langid.classify(city)[0] =='en':
#     row['city'] = row['city'].replace('Unknown', str(city))
#   else:
#     row['city'] = row['city']
#   return row
#   #return row['city']

In [ ]:
# #geolocator = Nominatim(user_agent="geoapiExercises")
# geolocator = Nominatim(user_agent="http")
# ### Function for finding out the city name from lattude and longitude
# def update_city(row):
    
#   Longitude = str(row['longitude'])
#   Latitude = str(row['latitude'])
#   location = geolocator.reverse(Latitude+","+Longitude, exactly_one=True)
#   address = location.raw['address']
#   city = address.get('state', '')

# ### Selecting the city name only in english language
#   if langid.classify(city)[0] =='en':
#     row['city'] = row['city'].replace('Unknown', str(city))
#   else:
#     row['city'] = row['city']
#   return row
#   #return row['city']

### Multiprocessing

In [ ]:
#pd.concat(result,axis =0).to_csv('Geocoder_city1.csv')

In [ ]:
pd.concat(result,axis =0)

In [ ]:
#unknown_city_data[0:5].apply(update_city , axis = 1)

In [ ]:
#multiprocessing.cpu_count()

In [ ]:
# def update_city2(df):
#     df2 = df.apply(update_city , axis = 1)
#     return df2

In [ ]:
# df1 = unknown_city_data[7010:7020]
# df2 = unknown_city_data[7020:7030]

In [ ]:
# if __name__ == '__main__':
#     df = unknown_city_data
#     p = Pool()
#     result = p.map(update_city2,[df1,df2
#                                  ])
#     p.close()
#     p.join()
#     print(result)

In [ ]:
#pd.concat(result,axis =0)

In [ ]:
#x.head(50)

In [ ]:
#pd.concat(result,axis =0).to_csv('Geocoder_data_part4.csv')

In [ ]:
# import multiprocessing
# import time

# def update_city2(df,queue):
#     df2 = df.apply(update_city , axis = 1)
#     queue.put(df)
#     #return df2

# queue = multiprocessing.SimpleQueue()
# tasks = [df1,df2]
                                 

# for task in tasks:
#     multiprocessing.Process(target=update_city2, args=(task, queue,)).start()

# for _ in tasks:
#     #print(pd.concat(queue.get(),axis = 0))
#     print(queue.get())

In [ ]:
# import multiprocessing
# import time

# def worker(x, queue):
#     time.sleep(1)
#     queue.put(x)

# queue = multiprocessing.SimpleQueue()
# tasks = range(10)

# for task in tasks:
#     multiprocessing.Process(target=worker, args=(task, queue,)).start()

# for _ in tasks:
#     print(queue.get())

In [ ]:
# def update_city(row):
#   Longitude = str(row['longitude'])
#   Latitude = str(row['latitude'])
#   location = geolocator.reverse(Latitude+","+Longitude, exactly_one=True)
#   address = location.raw['address']
#   city = address.get('state', '')
#   row['city'] = row['city'].replace('Unknown', str(city))
#   return row['city']

In [ ]:
# unknown_city_data.apply(update_city , axis = 1)

In [ ]:
# y = pd.DataFrame(x.apply(update_city , axis = 1),columns =['city'])
# y

In [ ]:
data.city.value_counts()

In [ ]:
((data_new.isna().sum()/data_new.shape[0])*100).sort_values(ascending= False)

In [ ]:
data_new.describe()

### Updating incorrect values

In [ ]:
### checking out the columns that contains 0 values for day and month 
data_new[(data_new.month == 0) | (data_new.day ==0)].head()

In [ ]:
### replacing month 0 values with month 12
data_new.month.replace(0,12 , inplace = True)

### replacing day 0 values with day 31
data_new.day.replace(0,31, inplace = True)

## Exploratory Analysis and Visualisation

In [ ]:
### total number of terrorist attacks per year basesd on success and not success
plt.rcParams['figure.figsize'] = (20, 5)
fig = px.histogram(data_new, 
                   x='year', 
                   color = 'success', 
                   color_discrete_sequence=['green', 'grey'], 
                   title='Number of terrorist attack per year')
fig.update_layout(bargap=0.1)
fig.update_xaxes(tickangle= -90)
fig.show()

In [ ]:
### Attacks per regions and top 20 countries where the terrorist attacks happended

plt.figure(figsize=(17, 10))
ax1 = plt.subplot(121)
colors = sns.color_palette('pastel')[0:10]+sns.color_palette('bright')[0:2]
data_new.region.value_counts().plot(kind='pie',autopct="%.1f%%",pctdistance= 1.09, colors = colors,labels = None)
                                              # ,rotatelabels=True,labeldistance=1)
ax1.legend(labels =data_new.region.value_counts().index, bbox_to_anchor = (1,1),loc = 2)
ax1.set_title('Total attacks per each region',size = 14)
ax2 = plt.subplot(122)
sns.barplot(x =data_new.groupby('country_name').eventid.count().sort_values(ascending=False)[:30].values , 
            y = data_new.groupby('country_name').eventid.count().sort_values(ascending=False)[:30].index )
ax2.set_title('Top 20 countries have higher number of attacks',size = 14)
plt.tight_layout()
plt.show()

In [ ]:
### Methode of attack
plt.rcParams['figure.figsize'] = (15, 8)
sns.countplot(x="attack_type", data= data_new, order=data_new.attack_type.value_counts().index)
plt.title('Methode of attack', size = 14)
plt.xticks(rotation=90)
plt.show()

In [ ]:
### types of victims of attack
plt.rcParams['figure.figsize'] = (20, 8)
sns.countplot(x="victim_cata", data= data_new, order=data_new.victim_cata.value_counts().index)
plt.title('Types of victims', size = 14)
plt.xticks(rotation=90)
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = (20, 5)
plt.title('Trends in Data (Understanding spikes)', size = 14)
data_new.groupby('year')['num_fatalities'].sum().plot()
data_new.groupby('year')['num_injured'].sum().plot()
data_new.groupby('year')['eventid'].count().plot()
data_new[data_new.property  == 1].groupby('year')['property'].sum().plot()

In [ ]:
plt.rcParams['figure.figsize'] = (12, 10)
sns.barplot(data_new['perpetrator_group'].value_counts()[1:25].values,data_new['perpetrator_group'].value_counts()[1:25].index)
plt.show()

In [ ]:
extended_attack_df = data_new[data_new['extended'] == 1]
non_extended_attack_df = data_new[~ (data_new['extended'] == 1)]

In [ ]:
plt.figure(figsize=(20, 5))
ax1 = plt.subplot(121)
sns.barplot(extended_attack_df['attack_type'].value_counts().values,extended_attack_df['attack_type'].value_counts().index, color='blue')
ax1.set_title('Attack extended more than 24 hours ',size = 14)
ax2 = plt.subplot(122)
sns.barplot(non_extended_attack_df['attack_type'].value_counts().values,non_extended_attack_df['attack_type'].value_counts().index, color='green')
ax2.set_title('Attack not extended more than 24 hours ',size = 14)
plt.tight_layout()
plt.show()

In [ ]:
data_new.columns

In [ ]:
### top 10 countries faced higher property damage
property_damage = data_new[data_new.property  == 1] 

In [ ]:
property_damage.groupby('country_name')['property'].sum().sort_values(ascending  =False)